# Problem set 4: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset4_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## Subset to the 17-1204 press release and take the string
pharma = str(doj.loc[doj["id"] == "17-1204"]["contents"])

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp
- `processtext` function here has an example of tokenizing and filtering to words where `.isalpha()` is true: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb
- Part of speech tagging section of this code: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb



In [4]:
## Restrict to alpha
preprocess = " ".join([text for text in pharma.split(" ") if text.isalpha()])

In [5]:
## Part of speech tagging
tokens = word_tokenize(preprocess) # Generate list of tokens
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens

In [6]:
## Sort Adjectives
adj = [token[0] for token in tokens_pos if token[1] == "JJ" or token[1] == "JJR" or token[1] == "JJS"]
adj_df = pd.DataFrame({"adjective": adj}).value_counts()
adj_df[:5]

adjective 
former        8
nationwide    4
addictive     3
other         3
opioid        3
dtype: int64

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

**Resources**:

- For parts A and B: named entity recognition part of this code: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb

In [7]:
## Part A
pharma_ents = nlp(pharma).ents

In [8]:
## Part B
entities = [ent.text for ent in pharma_ents if ent.label_ == "LAW"]
pd.DataFrame({"entity": entities})["entity"].unique()

array(['RICO', 'the Controlled Substances Act'], dtype=object)

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

RICO stands for Racketeer Influenced and Corrupt Organizations. Pharmaceutical kickbacks are illegal and considered fraud, which causes this case to fall under racketeering.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## Possible sentence lengths
regex = r'.*year.*'
poss_sentences = [ent.text for ent in pharma_ents if ent.label_ == "DATE" and re.match(regex, ent.text)]
poss_sentences

['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, `re.search` and `re.findall` examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_basicregex_solutions.ipynb

In [10]:
## Get 70 chars before and after
for search in poss_sentences:
    re.findall(".{70}" + search + ".{70}", pharma)

[' pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professio']

['it mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, ']

['ud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of ',
 'k Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine. Sentences are imposed by a ']

['olate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine. Sent']

['ud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of ',
 'k Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine. Sentences are imposed by a ']

#### E. It seems like the CEO is facing a max prison sentence of 20 years with 3 years of supervised release.

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## Subsetting
subset_topics = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]
doj_subset = doj[doj["topics_clean"].isin(subset_topics)]
print("doj_subset has " + str(len(doj_subset)) + " rows.")

doj_subset has 717 rows.


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
## SentimentIntensityAnalyzer
sent_obj = SentimentIntensityAnalyzer()

def get_sentiment(press_release):
    global sent_obj
    # remote entities
    press_ents = nlp(press_release).ents
    for ent in press_ents:
        press_release = press_release.replace(ent.text, "")
    #return the polarity_scores
    return sent_obj.polarity_scores(press_release)

In [13]:
## Executing the function
scores = [get_sentiment(release) for release in doj_subset["contents"]]
scores[:10]

[{'neg': 0.2, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931},
 {'neg': 0.129, 'neu': 0.804, 'pos': 0.066, 'compound': -0.9325},
 {'neg': 0.09, 'neu': 0.835, 'pos': 0.075, 'compound': -0.7579},
 {'neg': 0.121, 'neu': 0.798, 'pos': 0.081, 'compound': -0.9037},
 {'neg': 0.175, 'neu': 0.782, 'pos': 0.043, 'compound': -0.9864},
 {'neg': 0.148, 'neu': 0.8, 'pos': 0.053, 'compound': -0.987},
 {'neg': 0.146, 'neu': 0.78, 'pos': 0.074, 'compound': -0.9559},
 {'neg': 0.09, 'neu': 0.847, 'pos': 0.063, 'compound': -0.7783},
 {'neg': 0.103, 'neu': 0.838, 'pos': 0.059, 'compound': -0.9136},
 {'neg': 0.162, 'neu': 0.784, 'pos': 0.055, 'compound': -0.9801}]

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [14]:
## Add scores
doj_subset_wscore = doj_subset.copy()
new_columns = ["neg", "neu", "pos", "compound"]
for col in new_columns:
    doj_subset_wscore[col] = [s[col] for s in scores]
doj_subset_wscore = doj_subset_wscore.sort_values("neg", ascending = False)
doj_subset_wscore[["id", "contents", "neg"]][:2]

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.319
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.300


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [15]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby("topics_clean").agg({"compound":"mean"})

,compound
topics_clean,
Civil Rights,-0.087140
Hate Crimes,-0.935576
Project Safe Childhood,-0.653711


#### We might see veriation in the scores because some of them have more negative words than others. For example, the Hate Crimes topic has the most negative compound value. This makes sense given the fact that a press release convering Hate Crimes probabilty contains more negative words than that covering civil rights.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's code with topic modeling steps: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb

In [16]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]
# add custom to the default stopwords list
list_stopwords = stopwords.words("english") + custom_doj_stopwords

In [17]:
## your code defining a text processing function
snowball = SnowballStemmer("english")

def processing(release):
    release = release.lower().split()
    
    # remove stopwords, nonalpha, short words, and apply snoball stemmer
    nostop_release = [snowball.stem(word) for word in release
                      if word not in list_stopwords
                      and word.isalpha()
                      and len(word) >= 4]
    return " ".join(nostop_release)
    

In [18]:
## Executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(processing)

In [19]:
## Showing the examples
doj_subset_wscore[(doj_subset_wscore["id"] == "16-718") 
                   | (doj_subset_wscore["id"] == "16-217")][["id", "contents", "processed_text"]]

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",indict unseal mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick robert offic mississippi state beat includ punch throw victim marsher sturdiv charg violat right convict free cruel unusu sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri third deont charg along marsher sturdiv conspir cover indict alleg three offic submit fals report three lie marsher sturdiv face maximum sentenc year prison excess forc three offic face five year prison conspiraci fals statement year prison fals report indict mere defend presum innoc unless proven investig jackson cooper mississippi prosecut robert coleman northern mississippi dana mulhaus crimin marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a r

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the `create_dtm` function: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb


In [20]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [21]:
# Create dtm
dtm = create_dtm(list_of_strings= doj_subset_wscore.processed_text,
                      metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']])
dtm[:10]

,index,id,compound,topics_clean,aaron,abandon,abbat,abbi,abdomen,abduct,...,zack,zamd,zane,zealand,zealous,zeeman,zero,zobel,zone,zunggeemog
0,329,14-248,-0.9950,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,572,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11248,10-1194,-0.9990,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10635,11-1531,-0.9964,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,914,16-603,-0.9169,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,11903,18-70,-0.9908,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,3405,17-914,-0.9974,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,11645,10-448,-0.9989,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# prints out the top 10 words for the given subset
def get_topwords(subset):
    top_terms = subset[subset.columns[4:]].sum(axis = 0)
    res_terms = top_terms.sort_values(ascending = False)[:10]
    print(res_terms)

# rename because compound is a word that was found
column_names = dtm.columns.values
column_names[2] = 'compound_sen'
dtm.columns = column_names

# subset to top 5%
print("Top 10 for top 5% sentiment")
top_5_cutoff = dtm["compound_sen"].quantile(0.95)
top_5 = dtm[dtm["compound_sen"] > top_5_cutoff]
get_topwords(top_5)

# subset to bottom 5%
print()
print("Top 10 for bottom 5% sentiment:")
bottom_5_cutoff = dtm["compound_sen"].quantile(0.05)
bottom_5 = dtm[dtm["compound_sen"] < bottom_5_cutoff]
get_topwords(bottom_5)

# subset to each topic
for topic in subset_topics:
    print()
    print("Top 10 for " + topic + ":")
    get_topwords(dtm[dtm["topics_clean"] == topic])

Top 10 for top 5% sentiment
agreement     138
enforc        118
ensur         110
state          95
said           78
settlement     75
general        74
feder          72
communiti      72
work           71
dtype: int64

Top 10 for bottom 5% sentiment:
crime       130
offic       121
assault     116
hate        110
defend      109
feder        90
sentenc      88
prosecut     87
victim       84
guilti       83
dtype: int64

Top 10 for Civil Rights:
hous         588
offic        504
enforc       498
said         490
discrimin    476
feder        459
violat       444
alleg        389
general      386
state        347
dtype: int64

Top 10 for Hate Crimes:
prosecut    458
crime       445
hate        421
sentenc     420
feder       418
charg       406
said        404
defend      398
guilti      374
year        306
dtype: int64

Top 10 for Project Safe Childhood:
child       995
exploit     673
sexual      570
safe        475
project     472
crimin      404
prosecut    357
sentenc     329
ch

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [23]:
## Step 1: re-tokenize and store in list
text_raw_tokens = [wordpunct_tokenize(text) for text in 
                  doj_subset_wscore.processed_text]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)


## Step 3: filter out very rare and very common words
lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)


## Step 4: apply dictionary to TOKENIZED texts
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]


## Step 5: Estimating the model
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True,
                                         random_state = 1)

# print out top 15 words for each of the found topics
topics = ldamod.print_topics(num_words = 15)
counter = 0
for topic in topics:
    counter += 1
    print("Top 15 words from topic " + str(counter))
    print(topic)
    print()

Top 15 words from topic 1
(0, '0.020*"hous" + 0.016*"discrimin" + 0.013*"enforc" + 0.012*"said" + 0.011*"disabl" + 0.011*"alleg" + 0.010*"fair" + 0.010*"requir" + 0.010*"feder" + 0.010*"agreement" + 0.010*"inform" + 0.010*"violat" + 0.010*"general" + 0.010*"ensur" + 0.009*"state"')

Top 15 words from topic 2
(1, '0.043*"child" + 0.029*"exploit" + 0.026*"sexual" + 0.021*"safe" + 0.020*"project" + 0.018*"crimin" + 0.016*"prosecut" + 0.014*"sentenc" + 0.014*"children" + 0.011*"guilti" + 0.011*"investig" + 0.011*"state" + 0.010*"pornographi" + 0.010*"minor" + 0.010*"abus"')

Top 15 words from topic 3
(2, '0.017*"prosecut" + 0.016*"sentenc" + 0.016*"feder" + 0.016*"charg" + 0.015*"said" + 0.014*"defend" + 0.013*"guilti" + 0.013*"crime" + 0.012*"investig" + 0.011*"hate" + 0.011*"indict" + 0.011*"year" + 0.010*"victim" + 0.010*"violat" + 0.010*"general"')



## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/05_textasdata_partII_topicmodeling_solutions.ipynb
- If you're getting errors, use `shape`, `len`, and other commands to check the dimensionality of things at different steps 

In [24]:
## Get doc-level topic probabilities 
l=[ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
assert len(l) == len(doj_subset_wscore)

In [25]:
## Add those topic probabilities to the dataframe
top_topic = []
for prob in l:
    if prob[0][1] > prob[1][1] and prob[0][1] > prob[2][1]:
        top_topic.append(prob[0][0])
    if prob[1][1] > prob[0][1] and prob[1][1] > prob[2][1]:
        top_topic.append(prob[1][0])
    if prob[2][1] > prob[1][1] and prob[2][1] > prob[0][1]:
        top_topic.append(prob[2][0])
doj_subset_wscore["top_topic"] = top_topic

In [26]:
## Summarize the topic proportions for each of the topics_clean 
pd.crosstab(doj_subset_wscore.topics_clean, doj_subset_wscore.top_topic, normalize="index")


top_topic,0,1,2
topics_clean,,,
Civil Rights,0.659016,0.000000,0.340984
Hate Crimes,0.000000,0.000000,1.000000
Project Safe Childhood,0.006024,0.993976,0.000000


In [27]:
# Getting an example
doj_subset_wscore[doj_subset_wscore["topics_clean"] == "Civil Rights"][:2][["id", "contents", "topics_clean", "top_topic"]]

,id,contents,topics_clean,top_topic
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",Civil Rights,2
914,16-603,"Attorney General Loretta E. Lynch today released the following statement regarding the United States v. Dylann Roof: “Following the department’s rigorous review process to thoroughly consider all relevant factual and legal issues, I have determined that the Justice Department will seek the death penalty. The nature of the alleged crime and the resulting harm compelled this decision.”",Civil Rights,0


#### D. From the summary of top_topic mappings, it seems like we have found a pretty good match of Hate Crimes to 2, Project Safe Childhood to 1 and Civil Rights to 0. It seems like Civil Rights have a more difficult time mapping correctly and some examples are given above. Example "16-718" has a lot of violent content in its description, such as "kicking, punching and throwing the victim to the ground". We can logically see how this might be similar to the sort of language that would be used in a Hate Crime press release. Since language about violent/hate well defines Hate Crimes and language about children, explotation, etc well defines Project Safe Childhood, those topics were easy to map. There isn't really anything unique about Civil Rights which makes it more difficult and has it overlap with the Hate Crimes topic.

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [28]:
## function to create bigram doc
def create_bigram_onedoc(processed_text):
    as_list = processed_text.split(" ")
    bigram_list = []
    for w1, w2 in zip(as_list, as_list[1:]):
        bigram_list.append(w1 + "_" + w2)
    return " ".join(bigram_list)

doj_subset_wscore["processed_text_bigram"] = doj_subset_wscore["processed_text"].apply(create_bigram_onedoc)
doj_subset_wscore[doj_subset_wscore["id"] == "16-217"][["id", "processed_text", "processed_text_bigram"]]


,id,processed_text,processed_text_bigram
6727,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_announc announc_princip princip_deputi deputi_general general_vanita vanita_head head_wifredo wifredo_ferrer ferrer_southern southern_approv approv_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_resolv resolv_claim claim_stem stem_shoot shoot_conduct conduct_violent violent_crime crime_control control_enforc enforc_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_complianc complianc_settlement settlement_monitor monitor_independ independ_former former_polic polic_chief chief_jane jane_settlement settlement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_measur measur_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_said said_princip princip_deputi deputi_general general_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_said said_oversight oversight_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_citi citi_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_conduct conduct_attorney attorney_staff staff_special special_litig litig_section section_southern


C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [29]:
# creating dtm for the bigram
dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore.processed_text_bigram,
                      metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']])
dtm_bigram.head()

print("dtm dimensions: " + str(dtm.shape))
print("dtm_bigram dimensions: " + str(dtm_bigram.shape))

for topic in subset_topics:
    print()
    print("Top 10 for " + topic + ":")
    get_topwords(dtm_bigram[dtm_bigram["topics_clean"] == topic])

,index,id,compound,topics_clean,aaron_butler,aaron_constitut,aaron_convict,aaron_crimin,aaron_dylan,aaron_ford,...,zone_restrict,zone_right,zone_said,zone_student,zone_time,zone_tinley,zone_today,zone_varianc,zunggeemog_noel,zunggeemog_write
0,329,14-248,-0.9950,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,572,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11248,10-1194,-0.9990,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


dtm dimensions: (717, 6111)
dtm_bigram dimensions: (717, 63645)

Top 10 for Civil Rights:
fair_hous         225
deputi_general    221
princip_deputi    221
vanita_head       200
general_vanita    199
said_princip      186
announc_today     116
unit_state        106
consent_decre      92
act_general        80
dtype: int64

Top 10 for Hate Crimes:
hate_crime       299
plead_guilti     269
special_agent    117
year_prison      104
thoma_general     95
grand_juri        93
said_thoma        91
act_general       85
face_maximum      77
feder_hate        76
dtype: int64

Top 10 for Project Safe Childhood:
project_safe         472
child_exploit        266
child_pornographi    237
sexual_exploit       218
plead_guilti         188
child_sexual         177
exploit_obscen       175
safe_childhood       161
obscen_section       161
exploit_children     156
dtype: int64


#### D. The bigram matrix has more dimensions than the unigram matrix because there are many more unique bigrams that can be created as opposed to unigrams. For example, in a unigram model all words of "hello" are contained to that one word. In a bigram model, "hello" could yield an infinite amount of bigrams such as "hello_there" or "hello_john", etc.

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [30]:
# imports and tag
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(words=row["processed_text"],
                            tags=[row["id"]])
             for index, row in doj_subset_wscore.iterrows()]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=300, window=5, min_count=5, epochs=20)

In [31]:
# Compute pairwise similarity between all the press releases
max_sim = [0, 0]
ids = [[0, 0], [0, 0]]
for index, row in doj_subset_wscore.iterrows():
    similarities = model.docvecs.most_similar([model.infer_vector(row["processed_text"])])
    for i in range(2):
        if similarities[0][1] >= max_sim[i] and row["id"] != similarities[0][0]:
            max_sim[i] = similarities[0][1]
            ids[i][0] = row["id"]
            ids[i][1] = similarities[0][0]
            break

# score for max similarity and the id pairs
max_sim
ids

[0.8915703892707825, 0.877997875213623]

[['18-635', '16-672'], ['16-358', '16-672']]

In [34]:
counter = 0
for pair in ids:
    counter += 1
    print("Pair " + str(counter) + ":")
    print(doj_subset_wscore[(doj_subset_wscore["id"] == pair[0])][["id", "contents"]])
    print(doj_subset_wscore[(doj_subset_wscore["id"] == pair[1])][["id", "contents"]])
    print()

Pair 1:
           id  \
12862  18-635   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

#### The press releases pairs that are most similar seem to be press releases covering a similar crime. In both cases above, the press releases are about child pornography/exploitation cases which may explain the similarity result. Coincidentally both pairs involve the same press id 16-672.